# LiH molecule

## Origin of this method
 "Low rank representations for quantum simulation of electronic structure"
Mario Motta, Erika Ye, Jarrod R. McClean, Zhendong Li, Austin J. Minnich, Ryan Babbush, Garnet Kin-Lic Chan
https://arxiv.org/abs/1808.02625

The code is adapted from OpenFermion-Cirq Tutorial III: Low rank, arbitrary basis molecular simulations https://github.com/quantumlib/OpenFermion-Cirq/blob/master/examples/tutorial_3_arbitrary_basis_trotter.ipynb

## Low rank decomposition of the Coulomb operator

Tutorial I discusses methods for compiling single-particle basis transformations of fermionic operators in $O(N)$ depth on a linearly connected architecture.

Tutorial II discusses methods for compiling Trotter steps of electronic structure Hamiltonian in $O(N)$ depth on a linearly connected architecture when expressed in a basis diagonalizing the Coulomb operator so that
$$
H = \sum_{pq} T_{pq} a^\dagger_p a_q + \sum_{pq} V_{pq} a^\dagger_p a_p a^\dagger_q a_q.
$$

In Tutorial III both of those techniques are combined, along with some insights from electronic structure,
to simulate a Trotter step under the arbitrary basis two-body operator as
$$
\prod_{\ell=0}^{L-1} R_\ell \exp\left(-i\sum_{pq} f_{\ell p} f_{\ell q} a^\dagger_p a_p a^\dagger_q a_q\right) R_\ell^\dagger
$$
where we note that the operator in the exponential take the form of a diagonal Coulomb operator. Since we can implement the $R_\ell$ circuits in $O(N)$ depth (see Tutorial I) and we can implement Trotter steps under diagonal Coulomb operators in $O(N)$ layers of gates (see Tutorial II) we see that we can implement Trotter steps under arbitrary basis electronic structure Hamiltionians in $O(L N) = O(N^2)$ depth, and all on a linearly connected device.

## Example implementation: Trotter steps of LiH in molecular orbital basis

We will now use these techniques to implement Trotter steps for an actual molecule. We will focus on LiH at equilibrium geometry, since integrals for that system are provided with every OpenFermion installation. However, by installing [OpenFermion-PySCF](https://github.com/quantumlib/OpenFermion-PySCF) or [OpenFermion-Psi4](https://github.com/quantumlib/OpenFermion-Psi4) one can use these techniques for any molecule at any geometry. We will generate LiH in an active space consisting of 4 qubits. First, we obtain the Hamiltonian as an InteractionOperator.

In [1]:
import openfermion

# Set Hamiltonian parameters for LiH simulation in active space.
diatomic_bond_length = 1.45
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
active_space_start = 1
active_space_stop = 3

# Generate and populate instance of MolecularData.
molecule = openfermion.MolecularData(geometry, basis, multiplicity, description="1.45")
molecule.load()

# Get the Hamiltonian in an active space.
molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))
print(openfermion.get_fermion_operator(molecular_hamiltonian))

-6.7698132180879735 [] +
-0.7952726864779313 [0^ 0] +
0.24889540266275176 [0^ 0^ 0 0] +
-0.02307282640154995 [0^ 0^ 0 2] +
-0.023072826401549944 [0^ 0^ 2 0] +
0.005865992881900444 [0^ 0^ 2 2] +
0.24889540266275176 [0^ 1^ 1 0] +
-0.02307282640154995 [0^ 1^ 1 2] +
-0.023072826401549944 [0^ 1^ 3 0] +
0.005865992881900444 [0^ 1^ 3 2] +
0.04614563473199314 [0^ 2] +
-0.02307282640154995 [0^ 2^ 0 0] +
0.005865992881900456 [0^ 2^ 0 2] +
0.11412688446849813 [0^ 2^ 2 0] +
0.0027487522157917266 [0^ 2^ 2 2] +
-0.02307282640154995 [0^ 3^ 1 0] +
0.005865992881900456 [0^ 3^ 1 2] +
0.11412688446849813 [0^ 3^ 3 0] +
0.0027487522157917266 [0^ 3^ 3 2] +
0.24889540266275176 [1^ 0^ 0 1] +
-0.02307282640154995 [1^ 0^ 0 3] +
-0.023072826401549944 [1^ 0^ 2 1] +
0.005865992881900444 [1^ 0^ 2 3] +
-0.7952726864779313 [1^ 1] +
0.24889540266275176 [1^ 1^ 1 1] +
-0.02307282640154995 [1^ 1^ 1 3] +
-0.023072826401549944 [1^ 1^ 3 1] +
0.005865992881900444 [1^ 1^ 3 3] +
-0.02307282640154995 [1^ 2^ 0 1] +
0.00586599288

We see from the above output that this is a fairly complex Hamiltonian already. Next we will use the `simulate_trotter` function from Tutorial I, but this time with the built-in `LOW_RANK` Trotter step type, associated with these low rank techniques.

Setup the simulation environment

In [3]:
import cirq
import openfermioncirq
from openfermioncirq import trotter

# Trotter step parameters.
time = 1.
final_rank = 2

# Initialize circuit qubits in a line.
n_qubits = openfermion.count_qubits(molecular_hamiltonian)
qubits = cirq.LineQubit.range(n_qubits)

In the cell below, we compile the Trotter step with full rank so $L = N^2$ and depth is actually $O(N^3)$ and repeat the Trotter step multiple times to show that it actually converges to the correct result. Note that the rank of the Coulomb operators is asymptotically $O(N)$ but for very small molecules in small basis sets only a few eigenvalues can be truncated.

In [4]:
# Initialize a random initial state.
import numpy
random_seed = 8317
initial_state = openfermion.haar_random_vector(
    2 ** n_qubits, random_seed).astype(numpy.complex64)

# Trotter step paramaters.
n_steps = 3

# Compile the low rank Trotter step using OpenFermion-Cirq.
qubits = cirq.LineQubit.range(n_qubits)
circuit = cirq.Circuit(
    trotter.simulate_trotter(
            qubits, molecular_hamiltonian,
            time=time, n_steps=n_steps,
            algorithm=trotter.LOW_RANK),
    strategy=cirq.InsertStrategy.EARLIEST)

# Print circuit.
cirq.DropNegligible().optimize_circuit(circuit)
print(circuit.to_text_diagram(transpose=True))

0             1                     2                3
│             │                     │                │
Rz(π)         Rz(π)                 Rz(π)            Rz(π)
│             │                     │                │
│             PhISwap(0.25)─────────PhISwap(0.25)^-1 │
│             │                     │                │
PhISwap(0.25)─PhISwap(0.25)^0.081   │                │
│             │                     │                │
Rz(0.112π)    │                     PhISwap(0.25)────PhISwap(0.25)^-0.081
│             │                     │                │
│             PhISwap(0.25)─────────PhISwap(0.25)^-1 │
│             │                     │                │
│             Rz(0.112π)            │                Rz(0.056π)
│             │                     │                │
│             │                     Rz(0.056π)       │
│             │                     │                │
│             │                     │                │
│             │                  

In [5]:
# Numerically compute the correct circuit output.
import scipy
hamiltonian_sparse = openfermion.get_sparse_operator(molecular_hamiltonian)
exact_state = scipy.sparse.linalg.expm_multiply(
    -1j * time * hamiltonian_sparse, initial_state)

In [6]:
# Use Cirq simulator to apply circuit.
simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=qubits, initial_state=initial_state)
simulated_state = result.final_state

# Print final fidelity.
fidelity = abs(numpy.dot(simulated_state, numpy.conjugate(exact_state))) ** 2
print('Fidelity with exact result is {}.\n'.format(round(fidelity, 6)))


Fidelity with exact result is 0.999996.



In [7]:
# Compute next time step
exact_state = scipy.sparse.linalg.expm_multiply(
    -1j * time * hamiltonian_sparse, exact_state)

# Run simulator again on simulated state
result = simulator.simulate(circuit, qubit_order=qubits, initial_state=simulated_state)
simulated_state = result.final_state
fidelity = abs(numpy.dot(simulated_state, numpy.conjugate(exact_state))) ** 2
print('Fidelity with exact result is {}.\n'.format(round(fidelity, 6)))

Fidelity with exact result is 0.999985.



In [8]:
# Compute next time step
exact_state = scipy.sparse.linalg.expm_multiply(
    -1j * time * hamiltonian_sparse, exact_state)

# Run simulator again on simulated state
result = simulator.simulate(circuit, qubit_order=qubits, initial_state=simulated_state)
simulated_state = result.final_state
fidelity = abs(numpy.dot(simulated_state, numpy.conjugate(exact_state))) ** 2
print('Fidelity with exact result is {}.\n'.format(round(fidelity, 6)))

Fidelity with exact result is 0.999969.

